In [22]:
import pandas as pd

In [23]:
users = pd.read_csv("/Users/cemistanbullu/Desktop/VBO/github/datasets/users.csv")
purchases = pd.read_csv("/Users/cemistanbullu/Desktop/VBO/github/datasets/purchases.csv")

In [24]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [25]:
df = purchases.merge(users, how="inner", on="uid")

In [26]:
df.head()

,date,uid,price,reg_date,device,gender,country,age
0,2017-07-10,41195147,499,2017-06-26T00:00:00Z,and,M,BRA,17
1,2017-07-15,41195147,499,2017-06-26T00:00:00Z,and,M,BRA,17
2,2017-11-12,41195147,599,2017-06-26T00:00:00Z,and,M,BRA,17
3,2017-09-26,91591874,299,2017-01-05T00:00:00Z,and,M,TUR,17
4,2017-12-01,91591874,599,2017-01-05T00:00:00Z,and,M,TUR,17


In [27]:
agg_df = df.groupby(by=["country", "device", "gender", "age"]).agg({"price": "sum"}).sort_values("price", ascending=False)

In [28]:
agg_df.head()

price
country device gender age       
USA     and    M      15   61550
BRA     and    M      19   45392
DEU     iOS    F      16   41602
USA     and    F      17   40004
               M      23   39802

In [29]:
agg_df.reset_index(inplace=True)

In [30]:
mylabels = ["0_18", "19_23", "24_30", "31_40", "41_" + str(agg_df["age"].max())]
bins = [0, 18, 24, 31, 41, agg_df["age"].max()]

In [31]:
agg_df["age_cat"] = pd.cut(agg_df["age"], bins=bins, labels=mylabels)

In [32]:
agg_df.head()

,country,device,gender,age,price,age_cat
0,USA,and,M,15,61550,0_18
1,BRA,and,M,19,45392,19_23
2,DEU,iOS,F,16,41602,0_18
3,USA,and,F,17,40004,0_18
4,USA,and,M,23,39802,19_23


In [33]:
agg_df["customers_level_based"] = [row[0] + "_" + row[1].upper() + "_" + row[2] + "_" + row[5] for row in agg_df.values]

In [34]:
agg_df = agg_df[["customers_level_based", "price"]]

In [35]:
agg_df.head()

,customers_level_based,price
0,USA_AND_M_0_18,61550
1,BRA_AND_M_19_23,45392
2,DEU_IOS_F_0_18,41602
3,USA_AND_F_0_18,40004
4,USA_AND_M_19_23,39802


In [36]:
agg_df = agg_df.groupby("customers_level_based").agg({"price":"mean"})

In [37]:
agg_df.head()

,price
customers_level_based,
BRA_AND_F_0_18,26460.500000
BRA_AND_F_19_23,18221.666667
BRA_AND_F_24_30,11429.142857
BRA_AND_F_31_40,5686.250000
BRA_AND_F_41_75,4939.000000


In [38]:
agg_df = agg_df.reset_index()

In [39]:
segment = pd.qcut(agg_df["price"], 4, labels=["D", "C", "B", "A"])

In [40]:
agg_df["segment"] = segment

In [41]:
agg_df.head()

,customers_level_based,price,segment
0,BRA_AND_F_0_18,26460.500000,A
1,BRA_AND_F_19_23,18221.666667,A
2,BRA_AND_F_24_30,11429.142857,A
3,BRA_AND_F_31_40,5686.250000,B
4,BRA_AND_F_41_75,4939.000000,B
